# LFP example 


In [1]:

from bayes_window.generative_models import generate_fake_lfp
from bayes_window.visualization import plot_data, plot_data_slope_trials
from bayes_window import visualization
from importlib import reload


## Make fake data

In [2]:

df, df_monster, index_cols, _ = generate_fake_lfp()

In [3]:
reload(visualization)
visualization.plot_data(df=df,x='stim',y='Power',color='mouse')

alt.LayerChart(...)

In [4]:
c1=visualization.plot_data(df=df,x='stim',y='Log power').properties(width=60)
c1.facet('mouse')

alt.FacetChart(...)

In [5]:
reload(visualization)
visualization.plot_data_slope_trials(df=df,x='stim',y='Power',detail='i_trial').properties().facet(column='mouse')


alt.FacetChart(...)

In [6]:
reload(visualization)
c2=visualization.plot_data_slope_trials(df=df,x='stim',y='Log power',color=None,detail='i_trial')
(c1+c2).facet(column='mouse')

alt.FacetChart(...)

## Fit and plot

In [ ]:
from bayes_window import workflow
from bayes_window import models
from bayes_window import visualization
from importlib import reload
reload(workflow)
reload(visualization)
reload(models)

bw=workflow.BayesWindow(df,y='Log power', levels=('stim', 'mouse'))
bw.fit_slopes(add_data=False, model=models.model_hier_stim_one_codition,
              do_make_change='subtract', dist_y='normal')

In [ ]:
bw.plot_posteriors_slopes(add_box=False, independent_axes=True)

# Evaluate fit

In [9]:
from sklearn.metrics import roc_curve, auc
#roc_curve?
from joblib import Parallel, delayed

In [10]:
import numpy as np
def run_condition(true_slope):
    df, df_monster, index_cols, _ = generate_fake_lfp(mouse_response_slope=true_slope,
                                                     n_trials=20)
    bw=workflow.BayesWindow(df,y='Log power', levels=('stim', 'mouse'))
    bw.fit_slopes(add_data=False, model=models.model_hier_stim_one_codition,dist_y='student')
    return bw.data_and_posterior['lower HDI'].iloc[0]

true_slopes = np.hstack([np.zeros(10),np.arange(.3,.8,.01)])

#y_score=[run_condition(true_slope) for true_slope in true_slopes]

    

In [ ]:
y_score= Parallel(n_jobs=12,verbose=2)(delayed(run_condition)(true_slope) for true_slope in true_slopes)


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:   46.8s
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:  4.2min


In [ ]:
true_slopes,np.array(y_score)

In [ ]:
import matplotlib.pylab as plt
fpr, tpr, _ = roc_curve(true_slopes>0, np.array(y_score)>0)
print(fpr,tpr)
roc_auc = auc(fpr, tpr)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.plot(fpr, tpr,'.-', color='darkorange',
         label='ROC curve (area = %0.2f)' % roc_auc)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right");

